In [ ]:
# === STEP 32: Evaluasi Model Setelah Fine-Tuning ===
print("=== STEP 32: Evaluasi Model Setelah Fine-Tuning ===")

# Load model terbaik
from tensorflow.keras.models import load_model
best_model = load_model('best_rnn_model.h5')

# Evaluasi model terbaik
loss_finetuned, accuracy_finetuned = best_model.evaluate(X_test_rnn_smote, y_test_rnn_smote, verbose=0)
print("Fine-tuned RNN - Test Accuracy: {:.2f}%".format(accuracy_finetuned * 100))
print("Fine-tuned RNN - Test Loss: {:.4f}".format(loss_finetuned))

# Prediksi dengan model yang sudah di-fine-tune
y_pred_finetuned = best_model.predict(X_test_rnn_smote)
y_pred_finetuned_classes = np.argmax(y_pred_finetuned, axis=1)

# Classification report
print("\nClassification Report Fine-tuned Model:")
print(classification_report(y_test_rnn_smote, y_pred_finetuned_classes, 
                          target_names=label_encoder_smote.classes_))

In [ ]:
# === STEP 33: Fungsi Preprocessing untuk Testing ===
print("=== STEP 33: Fungsi Preprocessing untuk Testing ===")

def preprocess_single_text(text, slang_dict, stemmer, stopwords_set):
    """
    Fungsi untuk memproses satu kalimat text untuk prediksi
    """
    # Text cleaning
    if not isinstance(text, str):
        return ""
    
    processed_text = text.lower()
    processed_text = re.sub(r'http\S+|www\S+|https\S+', '', processed_text, flags=re.MULTILINE)
    processed_text = re.sub(r'\b\d+\b', '', processed_text, flags=re.MULTILINE)
    processed_text = re.sub(r'[:;=]-?[)(DP]', '', processed_text)
    processed_text = re.sub(r'[^\w\s]', '', processed_text)
    processed_text = re.sub(r'[^\x00-\x7F]+', '', processed_text)
    
    # Normalize slang
    processed_text = ' '.join([slang_dict.get(word, word) for word in processed_text.split()])
    
    # Remove stopwords
    processed_text = ' '.join([word for word in processed_text.split() if word not in stopwords_set])
    
    # Stemming
    processed_text = stemmer.stem(processed_text)
    
    return processed_text

def predict_sentiment_single(text, model, tokenizer, label_encoder, max_len, 
                           slang_dict, stemmer, stopwords_set):
    """
    Fungsi untuk memprediksi sentimen dari satu kalimat
    """
    # Preprocess text
    processed_text = preprocess_single_text(text, slang_dict, stemmer, stopwords_set)
    
    if not processed_text.strip():
        return "neutral", 0.0, [0.0, 0.0, 0.0]
    
    # Convert to sequence
    sequence = tokenizer.texts_to_sequences([processed_text])
    padded_sequence = pad_sequences(sequence, maxlen=max_len)
    
    # Predict
    prediction = model.predict(padded_sequence, verbose=0)
    predicted_class_idx = np.argmax(prediction[0])
    confidence = float(prediction[0][predicted_class_idx])
    
    # Get class name
    predicted_class = label_encoder.classes_[predicted_class_idx]
    
    return predicted_class, confidence, prediction[0].tolist()

In [ ]:
# === STEP 34: Testing dengan Contoh Kalimat ===
print("=== STEP 34: Testing dengan Contoh Kalimat ===")

# Contoh kalimat untuk testing
test_sentences = [
    "Saya sangat senang dengan pelayanan yang diberikan, sangat memuaskan!",
    "Pelayanannya buruk sekali, saya kecewa berat",
    "Pelayanan cukup baik, tidak ada yang istimewa",
    "Makanannya enak banget, pokoknya the best deh!",
    "Harganya terlalu mahal untuk kualitas seperti ini",
    "Tempatnya bersih dan nyaman untuk berkumpul"
]

print("="*80)
print("HASIL TESTING SENTIMENT ANALYSIS")
print("="*80)

for i, sentence in enumerate(test_sentences, 1):
    print(f"\n{i}. Kalimat Asli:")
    print(f"   '{sentence}'")
    
    # Preprocess untuk melihat hasil preprocessing
    processed = preprocess_single_text(sentence, slang_dict, stemmer, indo_stopwords)
    print(f"   Hasil Preprocessing: '{processed}'")
    
    # Predict sentiment
    predicted_sentiment, confidence, probabilities = predict_sentiment_single(
        sentence, best_model, tokenizer_smote, label_encoder_smote, 
        max_len_smote, slang_dict, stemmer, indo_stopwords
    )
    
    print(f"   Prediksi Sentimen: {predicted_sentiment.upper()}")
    print(f"   Confidence: {confidence:.4f} ({confidence*100:.2f}%)")
    print(f"   Probabilitas Detail:")
    for j, (label, prob) in enumerate(zip(label_encoder_smote.classes_, probabilities)):
        print(f"     - {label}: {prob:.4f} ({prob*100:.2f}%)")
    
    print("-" * 80)

In [ ]:
# === STEP 35: Interactive Testing ===
print("=== STEP 35: Interactive Testing ===")

def interactive_sentiment_test():
    """
    Fungsi untuk testing interaktif
    """
    print("\n" + "="*60)
    print("SENTIMENT ANALYSIS TESTING INTERAKTIF")
    print("="*60)
    print("Ketik 'quit' untuk keluar")
    
    while True:
        user_input = input("\nMasukkan kalimat untuk dianalisis: ").strip()
        
        if user_input.lower() == 'quit':
            print("Terima kasih! Testing selesai.")
            break
        
        if not user_input:
            print("Kalimat tidak boleh kosong!")
            continue
        
        # Predict sentiment
        predicted_sentiment, confidence, probabilities = predict_sentiment_single(
            user_input, best_model, tokenizer_smote, label_encoder_smote, 
            max_len_smote, slang_dict, stemmer, indo_stopwords
        )
        
        print(f"\n📝 Kalimat: '{user_input}'")
        print(f"🔮 Prediksi: {predicted_sentiment.upper()}")
        print(f"📊 Confidence: {confidence:.4f} ({confidence*100:.2f}%)")
        print(f"📈 Detail Probabilitas:")
        for label, prob in zip(label_encoder_smote.classes_, probabilities):
            bar = "█" * int(prob * 20)  # Visual bar
            print(f"   {label:8}: {bar:<20} {prob:.4f} ({prob*100:.2f}%)")

# Uncomment baris di bawah untuk menjala

In [ ]:
# === STEP 36: Perbandingan Performa Semua Model ===
print("=== STEP 36: Perbandingan Performa Semua Model ===")

# Kumpulkan semua akurasi
model_names = ['RNN Original', 'RNN + SMOTE', 'RNN Fine-tuned']
accuracies = [accuracy * 100, accuracy_smote * 100, accuracy_finetuned * 100]

# Visualisasi perbandingan
plt.figure(figsize=(12, 8))
bars = plt.bar(model_names, accuracies, color=['#1f77b4', '#ff7f0e', '#2ca02c'])

# Tambahkan nilai akurasi di atas bar
for bar, acc in zip(bars, accuracies):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
             f'{acc:.2f}%', ha='center', va='bottom', fontweight='bold', fontsize=12)

plt.ylabel('Accuracy (%)', fontsize=12)
plt.title('Perbandingan Performa Model RNN', fontsize=14, fontweight='bold')
plt.ylim(0, max(accuracies) + 10)
plt.grid(True, alpha=0.3, axis='y')

# Tambahkan garis untuk menunjukkan peningkatan
for i in range(len(accuracies)-1):
    improvement = accuracies[i+1] - accuracies[i]
    if improvement > 0:
        plt.annotate(f'+{improvement:.2f}%', 
                    xy=(i+0.5, max(accuracies[i], accuracies[i+1]) + 2),
                    ha='center', va='bottom', 
                    fontsize=10, color='green', fontweight='bold')

plt.tight_layout()
plt.show()

# Print summary
print("\n" + "="*60)
print("RINGKASAN PERFORMA MODEL")
print("="*60)
for name, acc in zip(model_names, accuracies):
    print(f"{name:15}: {acc:.2f}%")

print(f"\nPeningkatan Total: {accuracies[-1] - accuracies[0]:.2f}%")
print("="*60)

# === STEP 37: Simpan Model dan Preprocessing Tools ===
print("=== STEP 37: Simpan Model dan Preprocessing Tools ===")

import pickle

# Simpan tokenizer dan label encoder
with open('tokenizer_smote.pkl', 'wb') as f:
    pickle.dump(tokenizer_smote, f)

with open('label_encoder_smote.pkl', 'wb') as f:
    pickle.dump(label_encoder_smote, f)

# Simpan preprocessing tools
preprocessing_tools = {
    'slang_dict': slang_dict,
    'stemmer': stemmer,
    'stopwords': indo_stopwords,
    'max_len': max_len_smote
}

with open('preprocessing_tools.pkl', 'wb') as f:
    pickle.dump(preprocessing_tools, f)

print("Model dan tools preprocessing berhasil disimpan:")
print("- best_rnn_model.h5 (Model terbaik)")
print("- tokenizer_smote.pkl (Tokenizer)")
print("- label_encoder_smote.pkl (Label Encoder)")
print("- preprocessing_tools.pkl (Tools preprocessing)")

print("\n🎉 FINE-TUNING DAN TESTING SELESAI! 🎉")
print("Model siap digunakan untuk prediksi sentiment analysis!")